## Getting Started

## Initial Setup

In [13]:
!pip3 install google-cloud-aiplatform pandas scikit-learn seaborn --upgrade


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [14]:
# ! export GOOGLE_APPLICATION_CREDENTIALS="/sa-keys/labcamp-cred.json"

### Import libraries

In [16]:
import vertexai
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../sa-keys/labcamp_cred.json"
vertexai.init(project="cgai-project-1", location="us-central1")

import pandas as pd
import seaborn as sns
from IPython.display import Markdown, display
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import (
    TextGenerationModel,
    TextEmbeddingModel,
    ChatModel,
    InputOutputTextPair,
    CodeGenerationModel,
    CodeChatModel,
)

### Text generation with `text-bison@001`

#### Load model

In [17]:
generation_model = TextGenerationModel.from_pretrained("text-bison@001")

Create your first prompt and send it to the text generation model.

In [18]:
prompt = "What is a large language model?"

response = generation_model.predict(prompt=prompt)

print(response.text)

A large language model (LLM) is a type of artificial intelligence (AI) model that can understand and generate human language. LLMs are trained on massive datasets of text and code, and they can learn to perform a wide variety of tasks, such as translating languages, writing different kinds of creative content, and answering your questions in an informative way.

LLMs are still under development, but they have the potential to revolutionize many industries. For example, LLMs could be used to create more accurate and personalized customer service experiences, to help doctors diagnose and treat diseases, and to even write entire books and movies.




#### Try out your own prompt

In [19]:
prompt = """Create a numbered list of 10 items. Each item in the list should be a trend in the tech industry.

Each trend should be less than 5 words."""  # try your own prompt

response = generation_model.predict(prompt=prompt)

print(response.text)

1. **Artificial intelligence**
2. **Machine learning**
3. **Blockchain**
4. **Virtual reality**
5. **Augmented reality**
6. **5G**
7. **Edge computing**
8. **Internet of things**
9. **Self-driving cars**
10. **Wearables**


#### Prompt templates

There are many ways to implement prompt templates, and below is just one example using f-strings. (Importing variables into the prompt)

In [20]:
my_industry = "tech"  # try changing this to a different industry

response = generation_model.predict(
    prompt=f"""Create a numbered list of 10 items. Each item in the list should
    be a trend in the {my_industry} industry.

    Each trend should be less than 5 words."""
)

print(response.text)

1. **Artificial intelligence**
2. **Machine learning**
3. **Blockchain**
4. **Virtual reality**
5. **Augmented reality**
6. **5G**
7. **Edge computing**
8. **Internet of things**
9. **Self-driving cars**
10. **Wearables**


### Model parameters for `text-bison@001`

You can customize how the PaLM API behaves in response to your prompt by using the following parameters for `text-bison@001`:

 - `temperature`: higher means more "creative" responses
 - `max_output_tokens`: sets the max number of tokens in the output
 - `top_p`: higher means it will pull from more possible next tokens, based on cumulative probability
 - `top_k`: higher means it will sample from more possible next tokens

The section below covers each parameter and how to use them.

Testing that with `temperature=0`, the result is deterministic.

In [21]:
temp_val = 0.0
prompt_temperature = "Complete the sentence: As I prepared the picture frame, I reached into my toolkit to fetch my:"

response = generation_model.predict(
    prompt=prompt_temperature,
    temperature=temp_val,
)

print(f"[temperature = {temp_val}]")
print(response.text)

[temperature = 0.0]
As I prepared the picture frame, I reached into my toolkit to fetch my hammer.


Increase the temperature

In [22]:
temp_val = 1.0

response = generation_model.predict(
    prompt=prompt_temperature,
    temperature=temp_val,
)

print(f"[temperature = {temp_val}]")
print(response.text)

[temperature = 1.0]
As I prepared the picture frame, I reached into my toolkit to fetch my tape measure.


##### What is _max_output_tokens_?
`max_output_tokens` is the maximum number of tokens that can be generated in the response.

In [23]:
max_output_tokens_val = 5

response = generation_model.predict(
    prompt="List ten ways that generative AI can help improve the online shopping experience for users",
    max_output_tokens=max_output_tokens_val,
)

print(f"[max_output_tokens = {max_output_tokens_val}]")
print(response.text)

[max_output_tokens = 5]
1. **Personal


In [24]:
max_output_tokens_val = 500

response = generation_model.predict(
    prompt="List ten ways that generative AI can help improve the online shopping experience for users",
    max_output_tokens=max_output_tokens_val,
)

print(f"[max_output_tokens = {max_output_tokens_val}]")
print(response.text)

[max_output_tokens = 500]
1. **Personalized recommendations.** Generative AI can be used to create personalized recommendations for users based on their past purchases, browsing history, and other factors. This can help users find products that they are interested in and that are likely to be a good fit for them.
2. **Virtual try-on.** Generative AI can be used to create virtual try-on experiences for users, allowing them to see how different products would look on them before they buy them. This can help users make more informed decisions about their purchases and reduce the risk of buyer's remorse.
3. **Product design.** Generative AI can be used to help designers create new products that are more innovative and appealing to consumers. This can help businesses stay ahead of the competition and bring new products to market faster.
4. **Pricing.** Generative AI can be used to help businesses set prices for their products. This can help businesses maximize their profits and ensure that 

For easier reading, you can also render Markdown in Jupyter:

In [25]:
display(Markdown(response.text))

1. **Personalized recommendations.** Generative AI can be used to create personalized recommendations for users based on their past purchases, browsing history, and other factors. This can help users find products that they are interested in and that are likely to be a good fit for them.
2. **Virtual try-on.** Generative AI can be used to create virtual try-on experiences for users, allowing them to see how different products would look on them before they buy them. This can help users make more informed decisions about their purchases and reduce the risk of buyer's remorse.
3. **Product design.** Generative AI can be used to help designers create new products that are more innovative and appealing to consumers. This can help businesses stay ahead of the competition and bring new products to market faster.
4. **Pricing.** Generative AI can be used to help businesses set prices for their products. This can help businesses maximize their profits and ensure that they are not overcharging or undercharging for their products.
5. **Marketing.** Generative AI can be used to create marketing campaigns that are more effective and targeted. This can help businesses reach a wider audience and generate more leads.
6. **Customer service.** Generative AI can be used to provide customer service in a more efficient and personalized way. This can help businesses resolve customer issues faster and improve customer satisfaction.
7. **Fraud detection.** Generative AI can be used to detect fraud in online transactions. This can help businesses protect themselves from financial loss and ensure that their customers are safe.
8. **Inventory management.** Generative AI can be used to help businesses manage their inventory more effectively. This can help businesses avoid stockouts and ensure that they have the right products in stock when customers need them.
9. **Supply chain management.** Generative AI can be used to help businesses manage their supply chains more efficiently. This can help businesses reduce costs and improve delivery times.
10. **Logistics.** Generative AI can be used to help businesses optimize their logistics operations. This can help businesses reduce costs and improve the efficiency of their shipping and delivery processes.

#### The `top_p` parameter (range: 0.0 - 1.0, default 0.95)
The `top_p` parameter is used to control the diversity of the generated text. A higher `top_p` parameter value results in more "diverse" and "interesting" outputs

In [26]:
top_p_val = 0.0
prompt_top_p_example = (
    "Create a marketing campaign for jackets that involves blue elephants and avocados."
)

response = generation_model.predict(
    prompt=prompt_top_p_example, temperature=0.9, top_p=top_p_val
)

print(f"[top_p = {top_p_val}]")
print(response.text)

[top_p = 0.0]
```
Introducing the new Blue Elephant Avocado Jacket! This stylish and sustainable jacket is made from recycled materials and is perfect for any occasion.

The Blue Elephant Avocado Jacket is made from a soft, breathable fabric that will keep you warm and comfortable all day long. It features a relaxed fit and a variety of pockets to keep your essentials organized.

The Blue Elephant Avocado Jacket is also a fashion statement. The bold blue color and fun avocado print will turn heads wherever you go.

But what really sets the Blue Elephant Avocado Jacket apart is its sustainability. This jacket is made from recycled materials, and it is also certified by


In [27]:
top_p_val = 1.0

response = generation_model.predict(
    prompt=prompt_top_p_example, temperature=0.9, top_p=top_p_val
)

print(f"[top_p = {top_p_val}]")
print(response.text)

[top_p = 1.0]
**Introducing the new line of jackets from Blue Elephant Apparel!**

These jackets are made with a soft, durable fabric that will keep you warm and comfortable all winter long. They're also water-resistant, so you can stay dry even in the rain.

But what really sets these jackets apart is their unique design. Each jacket is embroidered with a blue elephant and an avocado, making it a stylish statement piece that will turn heads wherever you go.

**Order yours today and see the difference!**

**Here are some of the benefits of our jackets:**

* Made with a soft


#### The `top_k` parameter (range: 0.0 - 40, default 40)

##### What is _top_k_?
A`top_k` of 3 means that the next token is selected from the top 3 most probable tokens (using temperature). For each token selection step, the `top_k` tokens with the highest probabilities are sampled. Then tokens are further filtered based on `top_p` with the final token selected using temperature sampling.

In [28]:
prompt_top_k_example = "Write a 2-day itinerary for France."
top_k_val = 1

response = generation_model.predict(
    prompt=prompt_top_k_example, max_output_tokens=300, temperature=0.9, top_k=top_k_val
)

print(f"[top_k = {top_k_val}]")
print(response.text)

[top_k = 1]
Day 1:
* Morning: Start your day in Paris with a visit to the Eiffel Tower. Take the elevator to the top for stunning views of the city.
* Afternoon: After lunch, visit the Louvre Museum, one of the largest and most famous museums in the world. See some of the most iconic works of art, including the Mona Lisa and Venus de Milo.
* Evening: Enjoy a romantic dinner at a traditional French restaurant.

Day 2:
* Morning: Take a day trip to Versailles, the former home of French royalty. Explore the opulent palace and gardens.
* Afternoon: Visit the Palace of Fontainebleau, another former royal residence. See the beautiful architecture and gardens.
* Evening: Enjoy a leisurely dinner at a restaurant in Montmartre, a charming neighborhood in Paris.


In [29]:
top_k_val = 40

response = generation_model.predict(
    prompt=prompt_top_k_example,
    max_output_tokens=300,
    temperature=0.9,
    top_k=top_k_val,
)

print(f"[top_k = {top_k_val}]")
print(response.text)

[top_k = 40]
Day 1:
* Morning: Start your day with a visit to the Eiffel Tower. Take the elevator to the top for stunning views of Paris.
* Afternoon: Visit the Louvre Museum, one of the largest and most famous museums in the world. See works by some of the most famous artists in history, including Leonardo da Vinci, Michelangelo, and Rembrandt.
* Evening: Enjoy a romantic dinner at a cozy bistro in Montmartre.

Day 2:
* Morning: Take a stroll through the Tuileries Gardens, a beautiful park in the heart of Paris.
* Afternoon: Visit the Palace of Versailles, a UNESCO World Heritage Site. See the opulent royal apartments and gardens.
* Evening: Catch a show at the Moulin Rouge, a world-famous cabaret.


## Chat model with `chat-bison@001`

In [30]:
chat_model = ChatModel.from_pretrained("chat-bison@001")

chat = chat_model.start_chat()

print(
    chat.send_message(
        """
Hello! Can you write a 300 word abstract for a research paper I need to write about the impact of AI on society?
"""
    )
)

MultiCandidateTextGenerationResponse(text="Artificial intelligence (AI) is a branch of computer science that deals with the creation of intelligent agents, which are systems that can reason, learn, and act autonomously. AI has the potential to revolutionize many aspects of our lives, from the way we work and play to the way we interact with the world around us.\n\nThe impact of AI on society is a complex and multifaceted issue. On the one hand, AI has the potential to solve some of the world's most pressing problems, such as climate change, poverty, and disease. AI can also be used to create new products and services that improve our lives in ways that we can't even imagine.\n\nOn the other hand, AI also poses a number of risks to society. AI systems can be used to create autonomous weapons systems that could kill without human intervention. AI can also be used to create surveillance systems that track and monitor our every move.\n\nThe future of AI is uncertain. It is possible that AI

As shown below, the model should respond based on what was previously said in the conversation:

In [31]:
print(
    chat.send_message(
        """
Could you give me a catchy title for the paper?
"""
    )
)

MultiCandidateTextGenerationResponse(text='The Impact of Artificial Intelligence on Society', _prediction_response=Prediction(predictions=[{'candidates': [{'content': 'The Impact of Artificial Intelligence on Society', 'author': 'bot'}], 'groundingMetadata': [{}], 'citationMetadata': [{'citations': []}], 'safetyAttributes': [{'blocked': False, 'safetyRatings': [{'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.2, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Harassment'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech'}, {'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Sexually Explicit'}], 'scores': [0.1, 0.1, 0.3, 0.2, 0.1], 'categories': ['Derogatory', 'Finance', 'Health', 'Insult', 'Sexual']}]}], deployed_model_id='', metadata={'tokenMetadata': {'inputTokenCount': {'totalTokens': 430.0, 'totalBillab

### Advanced Chat model with the SDK
You can also provide a `context` and `examples` to the model. The model will then respond based on the provided context and examples. You can also use `temperature`, `max_output_tokens`, `top_p`, and `top_k`. These parameters should be used when you start your chat with `chat_model.start_chat()`.

For more information on chat models, please refer to the [documentation on chat model parameters](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/models#chat_model_parameters).

In [32]:
chat = chat_model.start_chat(
    context="My name is Ned. You are my personal assistant. My favorite movies are Lord of the Rings and Hobbit.",
    examples=[
        InputOutputTextPair(
            input_text="Who do you work for?",
            output_text="I work for Ned.",
        ),
        InputOutputTextPair(
            input_text="What do I like?",
            output_text="Ned likes watching movies.",
        ),
    ],
    temperature=0.3,
    max_output_tokens=200,
    top_p=0.8,
    top_k=40,
)
print(chat.send_message("Are my favorite movies based on a book series?"))

MultiCandidateTextGenerationResponse(text='Yes, they are.', _prediction_response=Prediction(predictions=[{'candidates': [{'content': 'Yes, they are.', 'author': '1'}], 'groundingMetadata': [{}], 'citationMetadata': [{'citations': []}], 'safetyAttributes': [{'blocked': False, 'safetyRatings': [{'probabilityScore': 0.0, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Harassment'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Sexually Explicit'}], 'scores': [0.1, 0.1, 0.1, 0.1], 'categories': ['Derogatory', 'Finance', 'Insult', 'Sexual']}]}], deployed_model_id='', metadata={'tokenMetadata': {'outputTokenCount': {'totalTokens': 5.0, 'totalBillableCharacters': 12.0}, 'inputTokenCount': {'totalTokens': 54.0, 'totalBillableCharacter

In [33]:
print(chat.send_message("When where these books published?"))

MultiCandidateTextGenerationResponse(text='The Lord of the Rings was published in 1954 and The Hobbit was published in 1937.', _prediction_response=Prediction(predictions=[{'candidates': [{'content': 'The Lord of the Rings was published in 1954 and The Hobbit was published in 1937.', 'author': 'bot'}], 'groundingMetadata': [{}], 'citationMetadata': [{'citations': []}], 'safetyAttributes': [{'blocked': False, 'safetyRatings': [{'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Dangerous Content'}, {'probabilityScore': 0.1, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Harassment'}, {'probabilityScore': 0.1, 'severityScore': 0.1, 'severity': 'NEGLIGIBLE', 'category': 'Hate Speech'}, {'probabilityScore': 0.2, 'severityScore': 0.0, 'severity': 'NEGLIGIBLE', 'category': 'Sexually Explicit'}], 'scores': [0.1, 0.1, 0.1, 0.2], 'categories': ['Derogatory', 'Insult', 'Religion & Belief', 'Sexual']}]}], deployed_model_id='', metadata={'tokenMetad

## Embedding model with `textembedding-gecko@001`

In [34]:
embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

embeddings = embedding_model.get_embeddings(["What is life?"])

for embedding in embeddings:
    vector = embedding.values
    print(f"Length = {len(vector)}")
    print(vector)

Length = 768
[0.010562753304839134, 0.04915031045675278, -0.022224493324756622, 0.0208794716745615, 0.024389723315835, 0.010366306640207767, 0.023919280618429184, 0.022391626611351967, -0.031569067388772964, 0.023535897955298424, -0.017047161236405373, -0.014345862902700901, 0.044956106692552567, 0.027327297255396843, -0.03314697742462158, -0.028214626014232635, -0.035373710095882416, -0.05229683220386505, 0.017105583101511, -0.03780610114336014, -0.07891207933425903, -0.01173518318682909, -0.01629730500280857, -0.04353305324912071, 0.013023999519646168, -0.10904901474714279, -0.0341256819665432, -0.0025329082272946835, -0.036971937865018845, -0.027775181457400322, 0.02332289144396782, 0.0052000475116074085, 0.005503748077899218, 0.0047489493153989315, -0.029920609667897224, 0.07563772797584534, 0.0007565636187791824, 0.03501711040735245, 0.02154686115682125, -0.000812096637673676, 0.06169590726494789, -0.024313345551490784, 0.03736764192581177, -0.0005869767046533525, -0.0228724572807

#### Embeddings and Pandas DataFrames

If your text is stored in a column of a DataFrame, you can create a new column with the embeddings with the example below.

In [35]:
text = [
    "i really enjoyed the movie last night",
    "so many amazing cinematic scenes yesterday",
    "had a great time writing my Python scripts a few days ago",
    "huge sense of relief when my .py script finally ran without error",
    "O Romeo, Romeo, wherefore art thou Romeo?",
]

df = pd.DataFrame(text, columns=["text"])
df

,text
0,i really enjoyed the movie last night
1,so many amazing cinematic scenes yesterday
2,had a great time writing my Python scripts a f...
3,huge sense of relief when my .py script finall...
4,"O Romeo, Romeo, wherefore art thou Romeo?"


Create a new column, `embeddings`, using the [apply](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) function in pandas with the embeddings model.

In [36]:
df

,text
0,i really enjoyed the movie last night
1,so many amazing cinematic scenes yesterday
2,had a great time writing my Python scripts a f...
3,huge sense of relief when my .py script finall...
4,"O Romeo, Romeo, wherefore art thou Romeo?"


In [37]:
df["embeddings"] = df.apply(
    lambda x: embedding_model.get_embeddings([x.text])[0].values, axis=1
)
df

,text,embeddings
0,i really enjoyed the movie last night,"[-0.015481960028409958, 0.010654411278665066, ..."
1,so many amazing cinematic scenes yesterday,"[-0.04475008696317673, 0.03850521519780159, -0..."
2,had a great time writing my Python scripts a f...,"[-0.002235855208709836, -0.009281235747039318,..."
3,huge sense of relief when my .py script finall...,"[-0.01665080338716507, 0.009947526268661022, 0..."
4,"O Romeo, Romeo, wherefore art thou Romeo?","[0.04232335463166237, -0.02867126651108265, 0...."


#### Comparing similarity of text examples using cosine similarity

In [38]:
cos_sim_array = cosine_similarity(list(df.embeddings.values))

# display as DataFrame
df = pd.DataFrame(cos_sim_array, index=text, columns=text)
df

# The close the result is to 1, the closer the results are

,i really enjoyed the movie last night,so many amazing cinematic scenes yesterday,had a great time writing my Python scripts a few days ago,huge sense of relief when my .py script finally ran without error,"O Romeo, Romeo, wherefore art thou Romeo?"
i really enjoyed the movie last night,1.000000,0.719884,0.631067,0.551936,0.459312
so many amazing cinematic scenes yesterday,0.719884,1.000000,0.622285,0.564447,0.524029
had a great time writing my Python scripts a few days ago,0.631067,0.622285,1.000000,0.738914,0.449386
huge sense of relief when my .py script finally ran without error,0.551936,0.564447,0.738914,1.000000,0.435728
"O Romeo, Romeo, wherefore art thou Romeo?",0.459312,0.524029,0.449386,0.435728,1.000000
